In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import numpy as np
import random
import re

data=pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

Number=10
weight=0.5
percent=1
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

data_1=data.copy()

data_1['Score_A']=0
data_1['Score_B']=0
data_1['Final__Score']=0
data_1=data_1.fillna(0)

Movie=input("Enter movie Name:")
regex = r'\b\w+\b'
list0=re.findall(regex,Movie)
list1=list0.copy()
count=len(list0)
for i in range(0,count):
    counter=0
    for j in range(0,7787):
        if (data_1.loc[j]["title"].find(list0[i])!=-1):
            counter+=1
    if (counter>(percent/100)*7787):
        list1.remove(list0[i])
count=len(list1)
if (count==0):
    list1=list0
for i in range(0,count):
    Movie=list1[i]
    data_1["Indexes"]= data_1["title"].str.find(Movie)
    data_1.loc[data_1["Indexes"] != -1, "Score_B"]+= 100/len(Movie)
    for j in range(0,7787):
        data_1.at[j,"Score_A"]+= levenshtein_ratio_and_distance(Movie,data_1.loc[j]["title"],ratio_calc=True)*100
        data_1.at[j,"Final_Score"]=data_1.loc[j]["Score_A"]*weight*(1/count)+data_1.loc[j]["Score_B"]*(1-weight)

data_2=data_1.sort_values("Final_Score",ascending=False).iloc[:,0:12].head(10)

print(data_2)
